# CS4487 Project: Image classification

## Regarding course project
* External training data is not allowed. Learning can only be performed on the 50000 training images in CIFAR10
* The **CIFAR10** dataset (*https://www.cs.toronto.edu/~kriz/cifar.html*) was comprised of 60, 000 32 × 32 color photographs of objects from 10 classes, with 6, 000 images per class. There are 50, 000 training images and 10, 000 test images.
* Submit your results at https://www.kaggle.com/c/cs4487cp/
* The second test set containing roughly 2000 images (refer to test_data.zip)

## Evaluation Criteria 

* ### Classification Accuaracy:
### $$ Acc(f, D) = \frac{1}{m}\sum_{i=1}^{m}\mathbb{I}[y^i = f(x^i)]$$

* ### File Format:
Upload a file in CSV format, such as:

|         **Index**         |         **Category**        |
|---------------------------|-----------------------------|
| 0                         | 8                           |
| 1                         | 4                           |
| 2                         | 6                           |
| 3                         | 1                           |
| ......                    | ......                      |
| 11999                     | 2                           |